In [1]:
!pip install audiomentations
!pip install timm

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 244 kB 863 kB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
#import cv2
#import audioread
#import logging
import os
import random
import time
import warnings
warnings.simplefilter('ignore')

from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import os,math,random
from itertools import islice

import librosa
import matplotlib.pyplot as plt

import timm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.utils.data as data

from pytorch_lightning.core import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import label_ranking_average_precision_score

import audiomentations as aaug

# LOGS

1. Trying new optimizer Admap
1. New adamp doesnt work
1. Increasing batchsize
1. Implement Imbalanced Dataset Sampler 
>     {Update: did not run}
>     Again trying 
    > Finally its working
1. Finally training 5 folds
1. Mixup Traning 5 folds
1. TPU Training 
> it didnt work

1. Trying larger model B4
1. Trying Pseudo labeling 
> did not try :)
1. Trying -Vision Transformer - Pytorch https://github.com/lucidrains/vit-pytorch

## Helper

In [3]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(1213)

In [4]:
PERIOD = 10
EPOCH = 40
DEVICE= torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
batch_size = 10
sample_rate = 48000
window_size = 2048
hop_size = 1024
mel_bins = 128
fmax = sample_rate/2.0
fmin = 40.0
classes_num = 24

data_dir = '../input/rfcx-species-audio-detection'
noise_dir = "../input/random-noise-audio"
train_tp_df = pd.read_csv(os.path.join(data_dir, 'train_tp.csv'))
#train_fp_df = pd.read_csv(os.path.join(data_dir, 'train_fp.csv'))
#data_test = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

OUTPUT_DIR = './'

## Dataset

In [5]:
def read_audio_test(recording_id):
    y, sr = librosa.load(
        os.path.join(data_dir, 'test' , recording_id + '.flac'),
        sr=None, mono=True, res_type="kaiser_fast"
    )

    effective_length = sample_rate*PERIOD
    # Split for enough segments to not miss anything
    segments = len(y) / effective_length
    segments = int(np.ceil(segments))
    
    features = []
    stride = 10 * sr
    y = np.stack([y[i:i+effective_length].astype(np.float32) for i in range(0, 60*sr+stride-effective_length, stride)])    
    #for i in range(0, segments):
    #    # Last segment going from the end
    #    if (i + 1) * effective_length > len(y):
    #         y = y[len(y) - effective_length:len(y)]
    #    else:
    #         y = y[i * effective_length:(i + 1) * effective_length]#

    #    features.append(y)

    return torch.tensor(y)

In [6]:
class MelSpecDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        df=None,
        num_classes=24,
        effective_length=sample_rate * PERIOD,
        waveform_transforms=None,
        is_train=False,
        data_dir=None,
    ):
        self.data_dir = data_dir
        self.is_train = is_train
        self.num_classes = num_classes
        self.df = df
        self.effective_length = effective_length
        self.waveform_transforms = waveform_transforms
        self.sampler = ImbalancedDatasetSampler

        self.length = len(self.df)
        self.data = list(train_full_df[["recording_id", "species_id"]].to_dict('index').values())
    def get_label(self, dataset, idx):
        
        return dataset.data[idx]["species_id"]

      
    def __getitem__(self, idx: int):
        
    
        recording_id = self.df["recording_id"].values[idx]

        prefix = 'train' if self.is_train else 'test'

        y, sr = librosa.load(
            os.path.join(self.data_dir, prefix , recording_id + '.flac'),
            sr=None, mono=True, res_type="kaiser_fast"
        )

        len_y = len(y)

        # y = butter_bandpass_filter(y, f_min, f_max, order=9)          
        t_min = float(self.df.t_min.values[idx]) * sr
        t_max = float(self.df.t_max.values[idx]) * sr
        
        # Positioning sound slice
        center = np.round((t_min + t_max) / 2)
        beginning = center - self.effective_length / 2
        if beginning < 0:
            beginning = 0
        beginning = np.random.randint( beginning , center)
        ending = beginning + self.effective_length
        if ending > len_y:
            ending = len_y
        beginning = ending - self.effective_length

        y = y[int(beginning):int(ending)].astype(np.float32)

        # Do the augmentations
        if self.waveform_transforms:
            y = self.waveform_transforms(y, sr)
        if self.is_train:
            target = np.zeros(self.num_classes, dtype=np.single)
            target[self.df.species_id.values[idx]] = 1
            return  y, target
        else:
            return  y
            
    def __len__(self):
        
        return self.length

train_full_df = train_tp_df

for index, row in train_full_df.iterrows():
    if fmin > float(row['f_min']):
        fmin = float(row['f_min'])
    if fmax < float(row['f_max']):
        fmax = float(row['f_max'])

# Get some safety margin
fmin = int(fmin * 0.9)
fmax = int(fmax * 1.1)



# Imbalance Data Sampler

In [7]:
#https://github.com/ufoym/imbalanced-dataset-sampler
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset, get_label):
        self.indices = list(range(len(dataset)))
        self.num_samples = len(self.indices)
            
        label_to_count = {}
        for idx in self.indices:
            label = dataset.get_label(dataset, int(idx))
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1

        weights = [1.0 / label_to_count[dataset.get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

## Mixup

In [8]:
class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size, device):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return torch.tensor(mixup_lambdas, requires_grad=False).float().to(device)

def do_mixup(x, mixup_lambda):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes 
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0 :: 2].transpose(0, -1) * mixup_lambda[0 :: 2] + \
        x[1 :: 2].transpose(0, -1) * mixup_lambda[1 :: 2]).transpose(0, -1)
    return out

In [9]:
class DFTBase(nn.Module):
    def __init__(self):
        """Base class for DFT and IDFT matrix"""
        super(DFTBase, self).__init__()

    def dft_matrix(self, n):
        (x, y) = np.meshgrid(np.arange(n), np.arange(n))
        omega = np.exp(-2 * np.pi * 1j / n)
        W = np.power(omega, x * y)
        return W

    def idft_matrix(self, n):
        (x, y) = np.meshgrid(np.arange(n), np.arange(n))
        omega = np.exp(2 * np.pi * 1j / n)
        W = np.power(omega, x * y)
        return W
    
    
class STFT(DFTBase):
    def __init__(self, n_fft=2048, hop_length=None, win_length=None, 
        window='hann', center=True, pad_mode='reflect', freeze_parameters=True):
        """Implementation of STFT with Conv1d. The function has the same output 
        of librosa.core.stft
        """
        super(STFT, self).__init__()

        assert pad_mode in ['constant', 'reflect']

        self.n_fft = n_fft
        self.center = center
        self.pad_mode = pad_mode

        # By default, use the entire frame
        if win_length is None:
            win_length = n_fft

        # Set the default hop, if it's not already specified
        if hop_length is None:
            hop_length = int(win_length // 4)

        fft_window = librosa.filters.get_window(window, win_length, fftbins=True)

        # Pad the window out to n_fft size
        fft_window = librosa.util.pad_center(fft_window, n_fft)

        # DFT & IDFT matrix
        self.W = self.dft_matrix(n_fft)

        out_channels = n_fft // 2 + 1

        self.conv_real = nn.Conv1d(in_channels=1, out_channels=out_channels, 
            kernel_size=n_fft, stride=hop_length, padding=0, dilation=1, 
            groups=1, bias=False)

        self.conv_imag = nn.Conv1d(in_channels=1, out_channels=out_channels, 
            kernel_size=n_fft, stride=hop_length, padding=0, dilation=1, 
            groups=1, bias=False)

        self.conv_real.weight.data = torch.Tensor(
            np.real(self.W[:, 0 : out_channels] * fft_window[:, None]).T)[:, None, :]
        # (n_fft // 2 + 1, 1, n_fft)

        self.conv_imag.weight.data = torch.Tensor(
            np.imag(self.W[:, 0 : out_channels] * fft_window[:, None]).T)[:, None, :]
        # (n_fft // 2 + 1, 1, n_fft)

        if freeze_parameters:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, input):
        """input: (batch_size, data_length)
        Returns:
          real: (batch_size, n_fft // 2 + 1, time_steps)
          imag: (batch_size, n_fft // 2 + 1, time_steps)
        """

        x = input[:, None, :]   # (batch_size, channels_num, data_length)

        if self.center:
            x = F.pad(x, pad=(self.n_fft // 2, self.n_fft // 2), mode=self.pad_mode)

        real = self.conv_real(x)
        imag = self.conv_imag(x)
        # (batch_size, n_fft // 2 + 1, time_steps)

        real = real[:, None, :, :].transpose(2, 3)
        imag = imag[:, None, :, :].transpose(2, 3)
        # (batch_size, 1, time_steps, n_fft // 2 + 1)

        return real, imag
    
    
class Spectrogram(nn.Module):
    def __init__(self, n_fft=2048, hop_length=None, win_length=None, 
        window='hann', center=True, pad_mode='reflect', power=2.0, 
        freeze_parameters=True):
        """Calculate spectrogram using pytorch. The STFT is implemented with 
        Conv1d. The function has the same output of librosa.core.stft
        """
        super(Spectrogram, self).__init__()

        self.power = power

        self.stft = STFT(n_fft=n_fft, hop_length=hop_length, 
            win_length=win_length, window=window, center=center, 
            pad_mode=pad_mode, freeze_parameters=True)

    def forward(self, input):
        """input: (batch_size, 1, time_steps, n_fft // 2 + 1)
        Returns:
          spectrogram: (batch_size, 1, time_steps, n_fft // 2 + 1)
        """

        (real, imag) = self.stft.forward(input)
        # (batch_size, n_fft // 2 + 1, time_steps)

        spectrogram = real ** 2 + imag ** 2

        if self.power == 2.0:
            pass
        else:
            spectrogram = spectrogram ** (power / 2.0)

        return spectrogram

    
class LogmelFilterBank(nn.Module):
    def __init__(self, sr=32000, n_fft=2048, n_mels=64, fmin=50, fmax=14000, is_log=True, 
        ref=1.0, amin=1e-10, top_db=80.0, freeze_parameters=True):
        """Calculate logmel spectrogram using pytorch. The mel filter bank is 
        the pytorch implementation of as librosa.filters.mel 
        """
        super(LogmelFilterBank, self).__init__()

        self.is_log = is_log
        self.ref = ref
        self.amin = amin
        self.top_db = top_db

        self.melW = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels,
            fmin=fmin, fmax=fmax).T
        # (n_fft // 2 + 1, mel_bins)

        self.melW = nn.Parameter(torch.Tensor(self.melW))

        if freeze_parameters:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, input):
        """input: (batch_size, channels, time_steps)
        
        Output: (batch_size, time_steps, mel_bins)
        """

        # Mel spectrogram
        mel_spectrogram = torch.matmul(input, self.melW)

        # Logmel spectrogram
        if self.is_log:
            output = self.power_to_db(mel_spectrogram)
        else:
            output = mel_spectrogram

        return output


    def power_to_db(self, input):
        """Power to db, this function is the pytorch implementation of 
        librosa.core.power_to_lb
        """
        ref_value = self.ref
        log_spec = 10.0 * torch.log10(torch.clamp(input, min=self.amin, max=np.inf))
        log_spec -= 10.0 * np.log10(np.maximum(self.amin, ref_value))

        if self.top_db is not None:
            if self.top_db < 0:
                raise ParameterError('top_db must be non-negative')
            log_spec = torch.clamp(log_spec, min=log_spec.max().item() - self.top_db, max=np.inf)

        return log_spec

In [10]:
class DropStripes(nn.Module):
    def __init__(self, dim, drop_width, stripes_num):
        """Drop stripes. 
        Args:
          dim: int, dimension along which to drop
          drop_width: int, maximum width of stripes to drop
          stripes_num: int, how many stripes to drop
        """
        super(DropStripes, self).__init__()

        assert dim in [2, 3]    # dim 2: time; dim 3: frequency

        self.dim = dim
        self.drop_width = drop_width
        self.stripes_num = stripes_num

    def forward(self, input):
        """input: (batch_size, channels, time_steps, freq_bins)"""

        assert input.ndimension() == 4

        if self.training is False:
            return input

        else:
            batch_size = input.shape[0]
            total_width = input.shape[self.dim]

            for n in range(batch_size):
                self.transform_slice(input[n], total_width)

            return input


    def transform_slice(self, e, total_width):
        """e: (channels, time_steps, freq_bins)"""

        for _ in range(self.stripes_num):
            distance = torch.randint(low=0, high=self.drop_width, size=(1,))[0]
            bgn = torch.randint(low=0, high=total_width - distance, size=(1,))[0]

            if self.dim == 2:
                e[:, bgn : bgn + distance, :] = 0
            elif self.dim == 3:
                e[:, :, bgn : bgn + distance] = 0


class SpecAugmentation(nn.Module):
    def __init__(self, time_drop_width, time_stripes_num, freq_drop_width, 
        freq_stripes_num):
        """Spec augmetation. 
        [ref] Park, D.S., Chan, W., Zhang, Y., Chiu, C.C., Zoph, B., Cubuk, E.D. 
        and Le, Q.V., 2019. Specaugment: A simple data augmentation method 
        for automatic speech recognition. arXiv preprint arXiv:1904.08779.
        Args:
          time_drop_width: int
          time_stripes_num: int
          freq_drop_width: int
          freq_stripes_num: int
        """

        super(SpecAugmentation, self).__init__()

        self.time_dropper = DropStripes(dim=2, drop_width=time_drop_width, 
            stripes_num=time_stripes_num)

        self.freq_dropper = DropStripes(dim=3, drop_width=freq_drop_width, 
            stripes_num=freq_stripes_num)

    def forward(self, input):
        x = self.time_dropper(input)
        x = self.freq_dropper(x)
        return x

In [11]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))
        model.bias.data.fill_(0)
    elif classname.find("BatchNorm") != -1:
        # get the number of the inputs
        n = model.in_features
        y = 1.0/np.sqrt(n)
        model.weight.data.normal_(-y, y)
        #model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for weight in model.parameters():
            if len(weight.size()) > 1:
                nn.init.orghogonal_(weight.data)
    elif classname.find("Linear") != -1:
        # get the number of the inputs
        n = model.in_features
        y = 1.0/np.sqrt(n)
        model.weight.data.normal_(-y, y)
        model.bias.data.zero_()


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output


class ConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weights()

    def init_weights(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):

        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x

class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

## EfficientNet Model

In [12]:
class PANNsEfficientNet(nn.Module):
    def __init__(self, sample_rate: int, window_size: int, hop_size: int,
                 mel_bins: int, fmin: int, fmax: int, classes_num: int,top_db=None):
        super().__init__()
        
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        self.interpolate_ratio = 30 # Downsampled ratio
        #self.apply_aug = apply_aug

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(
            n_fft=window_size,
            hop_length=hop_size,
            win_length=window_size,
            window=window,
            center=center,
            pad_mode=pad_mode,
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(
            sr=sample_rate,
            n_fft=window_size,
            n_mels=mel_bins,
            fmin=fmin,
            fmax=fmax,
            ref=ref,
            amin=amin,
            top_db=top_db,
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(
            time_drop_width=64,
            time_stripes_num=2,
            freq_drop_width=8,
            freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(mel_bins)

        self.base_model = timm.create_model('tf_efficientnet_b5_ns', pretrained=True)
        #self.model.head.fc = nn.Linear(n_features, n_class)
        #self.base_model = EfficientNet.from_pretrained('efficientnet-b4')
         
        #layers = list(base_model.children())[:-2]
        #self.encoder = nn.Sequential(*layers

        in_features = self.base_model.classifier.in_features
        
        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlock(in_features, classes_num, activation='sigmoid')
        self.init_weights()

        
    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        
    
    def preprocess(self, input_x, mixup_lambda=None):

        x = self.spectrogram_extractor(input_x)  # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)  # (batch_size, 1, time_steps, mel_bins)

        frames_num = x.shape[2]
        

        
        x = x.transpose(1, 3)
        
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.training:
            x = self.spec_augmenter(x)
        
        # Mixup on spectrogram
        if self.training  and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        return x, frames_num
        

    def forward(self, input_x):
        
        input_x, mixup_lambda = input_x
        """
        Input: (batch_size, data_length)"""
        
        x, frames_num = self.preprocess(input_x, mixup_lambda=mixup_lambda)
        
        
        # Output shape (batch size, channels, time, frequency)
        x = x.expand(x.shape[0], 3, x.shape[2], x.shape[3])
        
        x = self.base_model.forward_features(x)
        
        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.4, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.4, training=self.training)

        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, self.interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "framewise_logit":framewise_logit,
            "logit": logit,
            "clipwise_output": clipwise_output
        }

        return output_dict

## Ensemble Model

# Metrics

In [13]:
def _one_sample_positive_class_precisions(scores, truth):
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)

    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)

    retrieved_classes = np.argsort(scores)[::-1]

    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)

    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True

    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)

    precision_at_hits = retrieved_cumulative_hits[class_rankings[pos_class_indices]] / (
        1 + class_rankings[pos_class_indices].astype(np.float)
    )
    return pos_class_indices, precision_at_hits


def lwlrap(truth, scores):
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = _one_sample_positive_class_precisions(
            scores[sample_num, :], truth[sample_num, :]
        )
        precisions_for_samples_by_classes[
            sample_num, pos_class_indices
        ] = precision_at_hits

    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))

    per_class_lwlrap = np.sum(precisions_for_samples_by_classes, axis=0) / np.maximum(
        1, labels_per_class
    )
    return per_class_lwlrap, weight_per_class

class ImprovedPANNsLoss(nn.Module):
    def __init__(self, output_key="logit", weights=[1, 0.5]):
        super().__init__()

        self.output_key = output_key
        if output_key == "logit":
            self.normal_loss = nn.BCEWithLogitsLoss()
        else:
            self.normal_loss = nn.BCELoss()

        self.bce = nn.BCELoss()
        #self.bce = nn.BCEWithLogitsLoss()

        self.weights = weights

    def forward(self, input, target):
        input_ = input[self.output_key]
        target = target.float()

        framewise_output = input["framewise_output"]
        clipwise_output_with_max, _ = framewise_output.max(dim=1)

        normal_loss = self.normal_loss(input_, target)
        auxiliary_loss = self.bce(clipwise_output_with_max, target)

        return self.weights[0] * normal_loss + self.weights[1] * auxiliary_loss

#Modified focal loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, pred, target):

        pred_sigmoid = pred.sigmoid()
        target = target.type_as(pred)
        pt = (1 - pred_sigmoid) * target + pred_sigmoid * (1 - target)
        focal_weight = (self.alpha * target + (1 - self.alpha) *
                        (1 - target)) * pt.pow(self.gamma)
        loss = F.binary_cross_entropy_with_logits(
                        pred, target, reduction='none') * focal_weight
         
        return loss.mean()

class ImprovedFocalLoss(nn.Module):
    def __init__(self, weights=[1,1]):
        super().__init__()

        self.focal = FocalLoss()
        self.weights = weights

    def forward(self, input, target):
        input_ = input["logit"]
        target = target.float()

        framewise_output = input["framewise_logit"]
        clipwise_output_with_max, _ = framewise_output.max(dim=1)

        normal_loss = self.focal(input_, target)
        auxiliary_loss = self.focal(clipwise_output_with_max, target)

        return self.weights[0] * normal_loss + self.weights[1] * auxiliary_loss

### 5 Fold

In [14]:
args = {
    'sample_rate': sample_rate,
    'window_size': window_size,
    'hop_size': hop_size,
    'mel_bins': mel_bins,
    'fmin': fmin, 
    'fmax': fmax,
    'classes_num': classes_num
}

In [15]:
class LightModel(LightningModule):
    def __init__(self, model,fold_id = 0, data_dir=data_dir,
                 batch_size=batch_size, num_workers=4, DEVICE=DEVICE, mixup = False):
        super().__init__()
        self.model = model
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.DEVICE = DEVICE
        self.loss_fn = ImprovedPANNsLoss()
        #self.loss_fn = ImprovedFocalLoss()
        self.fold_id = fold_id
        self.mixup = mixup
        #self.train_df, self.val_df = train_df, valid_df

    def forward(self, batch):
        return self.model(batch)
   
    def training_step(self, batch, batch_idx):
        y, target = batch
        
        if self.mixup:
            mixup_augmenter = Mixup(1.)
            mix_y = mixup_augmenter.get_lambda(batch_size=self.batch_size, device = DEVICE)
            mix_target = do_mixup(target,mixup_augmenter.get_lambda(batch_size=self.batch_size, device = DEVICE))
            output = model((y,mix_y))
            bceLoss = self.loss_fn(output, mix_target)
        else:
            output = model((y,None))
            bceLoss = self.loss_fn(output, target)
        loss = bceLoss
        return loss
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        logs = {"train_loss": avg_loss}
    
    def validation_step(self, batch, batch_idx):
        y, target = batch
        output = model((y,None))
        bceLoss = self.loss_fn(output, target)
        loss = bceLoss
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        p = torch.sigmoid(output['clipwise_output'])
        score_class, weight = lwlrap(target.cpu().numpy(), p.cpu().numpy())
        score = (score_class * weight).sum()
        self.log("Lwlrap_epoch", score, on_epoch=True, prog_bar=True)

    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, amsgrad = True, weight_decay = 1e-2)        
        #optimizer = AdamP(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10,)
        lr_scheduler = {"scheduler": scheduler }
        return [optimizer], [lr_scheduler]
    
    
    def prepare_data(self):
        
        image_source = train_tp_df[['recording_id', 'species_id']].drop_duplicates()

        # get lists for image_ids and sources
        image_ids = image_source['recording_id'].to_numpy()
        sources = image_source['species_id'].to_numpy()

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        split = skf.split(image_ids, sources) # second arguement is what we are stratifying by

        select = self.fold_id #Use fold
        train_ix, val_ix = next(islice(split, select, select+1))

            # translate indices to ids
        train_ids = image_ids[train_ix]
        val_ids = image_ids[val_ix]

        # create corresponding dfs
        self.train_df = train_tp_df[train_tp_df['recording_id'].isin(train_ids)]
        self.val_df = train_tp_df[train_tp_df['recording_id'].isin(val_ids)]
                                                
    def train_dataloader(self):
        sampler = ImbalancedDatasetSampler(MelSpecDataset(data_dir = self.data_dir,
                                                          df = self.train_df,
                                                          is_train=True),
                                           MelSpecDataset.get_label)
        train_aug = aaug.Compose([
            aaug.AddGaussianNoise(p=0.5),
            aaug.AddGaussianSNR(p=0.5),
            aaug.Gain(p=0.5),
            aaug.AddBackgroundNoise(sounds_path = noise_dir, p=0.5)
            #aaug.Normalize(p=0.3)
            #aaug.Shift(p=0.2)
            #aaug.PitchShift(min_semitones=-4, max_semitones=4, p=0.3),
            #aaug.TimeStretch(p = 0.3)
       ])

        return DataLoader(
            MelSpecDataset(
                data_dir=self.data_dir,
                df=self.train_df, 
                is_train=True,
                waveform_transforms=train_aug,
            ),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=True,
            drop_last=True,
            sampler = sampler
        )

    def val_dataloader(self):
        return DataLoader(
            MelSpecDataset(
                data_dir=self.data_dir,
                df=self.val_df, 
                is_train=True
            ), 
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=True,
            drop_last=False
        )

# Models

In [16]:
models = [
    PANNsEfficientNet(**args),
    PANNsEfficientNet(**args),
    PANNsEfficientNet(**args),
    PANNsEfficientNet(**args),
    PANNsEfficientNet(**args),
]

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b5_ns-6f26d0cf.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b5_ns-6f26d0cf.pth


# Do Training

In [17]:
for k in range(5):
    
    print("Current  Fold:",k)

    model = LightModel(models[k], fold_id = k, mixup = False,batch_size = 10)

    checkpoint_callback = ModelCheckpoint(
        filepath=OUTPUT_DIR,
        monitor='Lwlrap_epoch',
        mode='max',
        prefix=f'PANNsEfficientNetb4_fold{k}_',
        save_last = True,
        save_top_k = 1,
        save_weights_only=False,
        period = 1,
        verbose = False
    )

    logger = TensorBoardLogger(
        save_dir=OUTPUT_DIR,
        name=f'lightning_logs{k}'
    )

    trainer = Trainer(
        max_epochs=EPOCH,
        logger=logger,
        checkpoint_callback=checkpoint_callback,
        gpus=int(torch.cuda.is_available()),      
    )    
    trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current  Fold: 0



  | Name    | Type              | Params
----------------------------------------------
0 | model   | PANNsEfficientNet | 39.0 M
1 | loss_fn | ImprovedPANNsLoss | 0     


Saving latest checkpoint...


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Saving latest checkpoint...


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current  Fold: 4



  | Name    | Type              | Params
----------------------------------------------
0 | model   | PANNsEfficientNet | 39.0 M
1 | loss_fn | ImprovedPANNsLoss | 0     


Saving latest checkpoint...


In [18]:
print('Success!!!')

Success!!!
